# Results Neurocomputing/ESANN 2024

This notebook contains the code to analyse the results of the 
Neurocomputing/ESANN 2024 paper, and it is responsible for generating
the figures and tables in the paper.

The notebook is organised as follows:

1. The first section contains imports, constants, helper functions and load the 
    data.

2. We show that the dict-wisard has competitive performance with the 
    classical machine learning algorithms.

## 1. General constants, hhelper functions, and data loading

Imports, global constants and packages' configuration.

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
from typing import List, Union
import plotly.graph_objects as go
import plotly.express as px

In [3]:
# Configs
# pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: "%.4f" % x)

# ---------- Paths -------------
# -- Inputs
datasets_info_path = Path("../experiments/datasets_info.json")

results_path = Path("../experiments_2/results_tune/results.csv")
results_sklearn_path = Path(
    "../experiments/results_sklearn/results_sklearn.csv"
)

# --- Ouptuts
latex_tables_path = Path("tables")
figures_path = Path("figures")

Useful functions

In [4]:
def write_figure(
    filename: str, fig: go.Figure, path: Union[Path, str] = figures_path
):
    """Write a Figure to a file.

    Parameters
    ----------
    filename : str
        The name of the file to write to.
    fig : go.Figure
        The plotly figure object.
    path : Union[Path, str], optional
        The path where the file will be stored, by default figures_path
    """
    path = Path(path)
    path.mkdir(exist_ok=True, parents=True)
    fname = path / filename
    fig.write_image(fname)

    print(f"Figure written to: {fname}")
    print(f"Filename   :", filename)
    print(f"Latex label:", filename.replace(".pdf", ""))


def write_latex_table(
    filename: str, table: str, path: Union[Path, str] = latex_tables_path
):
    """Write a latex table to a file.

    Parameters
    ----------
    filename : str
        The name of the file to write to.
    table : str
        The table, as a string.
    path : Union[Path, str], optional
        The path where the file will be stored, by default latex_tables_path
    """
    path = Path(path)
    path.mkdir(exist_ok=True, parents=True)
    fname = path / filename
    with fname.open("w") as f:
        f.write(
            "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n"
        )
        f.write(
            "%% WARNING: DO NOT CHANGE THIS FILE. IT IS GENERATED AUTOMATICALLY %\n"
        )
        f.write(
            "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n"
        )
        f.write(table)
    print(f"Table written to: {fname}")


def aggregate_mean_std(
    df: pd.DataFrame,
    group_by: List[str],
    keys_to_aggregate: List[str],
) -> pd.DataFrame:
    """Group and aggregate columns of a dataframe, using mean and std.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe to aggregate.
    group_by : List[str]
        The columns used to group the dataframe.
    keys_to_aggregate : List[str]
        The column names that will be aggregated.

    Returns
    -------
    pd.DataFrame
        A dataframe with the aggregated values.

    Example
    -------
    >>> df = pd.DataFrame({
    ...     "a": [1, 1, 2, 2],
    ...     "b": [1, 2, 3, 4],
    ...     "c": [5, 6, 7, 8],
    ... })
    >>> aggregate_mean_std(df, ["a"], ["b", "c"])
       a    b  b_std    c  c_std
    0  1 1.500  0.707 5.500  0.707
    1  2 3.500  0.707 7.500  0.707

    """
    x = (
        df.groupby(group_by)[keys_to_aggregate]
        .agg("mean")
        .join(
            df.groupby(group_by)[keys_to_aggregate].agg("std"), rsuffix="_std"
        )
    )
    return x.reset_index()

### Read inputs and create a full dataframe

1. Read the datasets specifications (`dataset_info`)
2. Read the wisard results (`wisard_results`)
3. Read the sklearn results (`sklearn_results`)
4. Create a results dataframe, mergind dataset_info, wisard_results and sklearn_results

#### Dataset information

In [4]:
# Datasets information
datasets_info = pd.read_json(datasets_info_path, orient="index").reset_index(drop=True)
datasets_info.rename(columns={"name": "dataset_name"}, inplace=True)
datasets_info.head(n=3)

,dataset_name,size,features,num_classes,train_size,test_size,balanced,metric
0,breast_cancer,141416,30,3,398,171,False,f1 weighted
1,dry_bean,1773910,16,7,10888,2723,False,f1 weighted
2,glass,17413,9,24,149,65,False,f1 weighted


In [5]:
info = datasets_info[
    ["dataset_name", "features", "size", "num_classes", "balanced"]
]
info["size"] = info["size"] / 1024

info = info.rename(
    columns={
        "dataset_name": "Dataset",
        "features": "Features",
        "size": "Size (KB)",
        "num_classes": "Classes",
        "balanced": "Is Balanced?",
    }
)

latex_str = info.to_latex(
    index=False,
    escape=True,
    caption="Datasets information",
    label="tab:datasets_info",
    float_format="%.2f",
)

write_latex_table("datasets_info.tex", latex_str)

/tmp/ipykernel_361841/3092149693.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info["size"] = info["size"] / 1024


Table written to: tables/datasets_info.tex


/tmp/ipykernel_402591/3092149693.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info["size"] = info["size"] / 1024
/tmp/ipykernel_402591/3092149693.py:16: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_str = info.to_latex(


#### Wisard results

Read and parse wisard result to `wisard_results` dataframe.

**Note**: The `wisard_results` already has aggregated results for each dataset.

In [6]:
def parse_wisard_config_name(row) -> str:
    """Given a row, parse the name of configuration.

    Parameters
    ----------
    row : pd.Series
        The row of the dataframe.

    Returns
    -------
    str
        The name of the configuration.
    """

    names = []
    if not pd.isna(row["num_hitters"]):
        names.append(f"NR: {row['num_hitters']}")
    if not pd.isna(row["width"]):
        names.append(f"W: {row['width']}")
    if not pd.isna(row["depth"]):
        names.append(f"D: {row['depth']}")
    if not pd.isna(row["capacity"]):
        names.append(f"C: {row['capacity']}")
    if not pd.isna(row["bucket_size"]):
        names.append(f"BS: {row['bucket_size']}")
    if not pd.isna(row["threshold"]):
        names.append(f"T: {row['threshold']}")
    if not pd.isna(row["est_elements"]):
        names.append(f"EST: {row['est_elements']}")
    if not pd.isna(row["false_positive_rate"]):
        names.append(f"FPR: {row['false_positive_rate']}")

    if names:
        names = ", ".join(names)
        return f"{row['ram']} ({names})"
    else:
        return row["ram"]


# --- Read results and add a column with the name of the configuration ---
wisard_results = pd.read_csv(results_path).drop_duplicates()

# --- Add useful columns ---
wisard_results["tuple_size"] = (
    wisard_results["resolution"] / wisard_results["tuple_resolution_factor"]
)
wisard_results["config_name"] = wisard_results.apply(
    parse_wisard_config_name, axis=1
)

# --- Select the columns of interest ---
wisard_results = wisard_results[
    [
        "dataset_name",
        "config_name",
        "test_accuracy_mean",
        "test_accuracy_std",
        "test_f1 weighted_mean",
        "test_f1 weighted_std",
        "test_model size_mean",
        "test_model size_std",
        "test_ties_mean",
        "test_ties_std",
        "tuple_size",
        "encoder",
        "resolution",
        "bleach",
        "rams per discriminator",
        "ram",
    ]
]

# --- Rename columns ---
wisard_results = wisard_results.rename(
    columns={
        "dataset_name": "dataset",
        "ram": "model",
        "test_ties_mean": "ties",
        "test_ties_std": "ties_std",
        "test_accuracy_mean": "accuracy",
        "test_accuracy_std": "accuracy_std",
        "test_f1 weighted_mean": "f1",
        "test_f1 weighted_std": "f1_std",
        "test_model size_mean": "model_size",
        "test_model size_std": "model_size_std",
    }
)

# --- Add model column and drop duplicates ---
wisard_results["model"] = "wisard"
wisard_results.drop_duplicates(inplace=True)

wisard_results.sample(n=2)

,dataset,config_name,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std,ties,ties_std,tuple_size,encoder,resolution,bleach,rams per discriminator,model
29608,breast_cancer,"CountMinSketch (W: 337.0, D: 1.0)",0.3743,0.0000,0.2039,0.0000,81840.0000,0.0000,140.0000,0.0000,4.0000,thermometer,4,3,30,wisard
49365,image_segmentation,"CountMinSketch (W: 145.0, D: 3.0)",0.4762,0.0337,0.4312,0.0417,233548.0000,0.0000,7.0000,1.6330,47.0000,distributive-thermometer,47,14,19,wisard


#### Scikit Learn results

Read and parse sklearn result to `sklearn_results` dataframe.

In [7]:
# Read sklearn results and aggregate multiple runs
sklearn_results = pd.read_csv(results_sklearn_path).drop_duplicates()

# Aggregate metric for multiple runs
sklearn_results = aggregate_mean_std(
    df=sklearn_results,
    group_by=[
        "model",
        "model kwargs",
        "dataset name",
        "experiment name",
    ],
    keys_to_aggregate=[
        "accuracy",
        "f1 weighted",
        "f1 macro",
        "f1 micro",
        "train time",
        "predict time",
        "model size",
    ],
)

# Select columns of interest
sklearn_results = sklearn_results[
    [
        "dataset name",
        "model",
        "model kwargs",
        "accuracy",
        "accuracy_std",
        "f1 weighted",
        "f1 weighted_std",
        "model size",
        "model size_std",
    ]
]

# Rename columns
sklearn_results.rename(
    columns={
        "dataset name": "dataset",
        "model kwargs": "config_name",
        "f1 weighted": "f1",
        "f1 weighted_std": "f1_std",
        "model size": "model_size",
        "model size_std": "model_size_std",
    },
    inplace=True,
)

# Add config_name (name of the model)
sklearn_results["config_name"] = ""

sklearn_results.sample(n=2)

,dataset,model,config_name,accuracy,accuracy_std,f1,f1_std,model_size,model_size_std
70,wine,mlp,,0.8889,0.0000,0.8884,0.0000,49203.0000,0.0000
25,motion_sense,knn,,0.5049,0.0000,0.4933,0.0000,9860342.0000,0.0000


#### Merge wisard results, sklearn results, and dataset information

Create a full dataframe with all the information named  `base_results`.

In [8]:
# results is a dataframe with the results of both wisard (only dict ram) and sklearn
# temp = wisard_results.loc[wisard_results[(wisard_results["config_name"] == "Dict")].groupby("dataset")["accuracy"].idxmax()]
# results = pd.concat([temp, sklearn_results], ignore_index=True)
base_results = pd.concat([wisard_results, sklearn_results], ignore_index=True)
base_results = pd.merge(
    base_results,
    datasets_info,
    left_on="dataset",
    right_on="dataset_name",
    how="left",
    validate="many_to_one",
)

base_results.rename(columns={"metric": "metric_to_use"}, inplace=True)

# add a metric column to results. If metric_to_use is f1 weighted, copy the f1 column to metric, else copy the accuracy column. Do it for each row
base_results["metric"] = base_results.apply(
    lambda row: row["accuracy"]
    if row["metric_to_use"] == "accuracy"
    else row["f1"],
    axis=1,
)
base_results["metric_std"] = base_results.apply(
    lambda row: row["accuracy_std"]
    if row["metric_to_use"] == "accuracy"
    else row["f1_std"],
    axis=1,
)

# Rename some columns
base_results.rename(
    columns={
        "size": "dataset_size",
    },
    inplace=True,
)

# Remove dataset_name column
base_results.drop(columns=["dataset_name"], inplace=True)

# We use metric column to compare models, instead of accuracy or f1

# Put dataset, model, config_name, metric, metric_std, model_size, model_size_std columns first
cols = base_results.columns.tolist()
cols = [
    "dataset",
    "model",
    "config_name",
    "metric",
    "metric_std",
    "model_size",
    "model_size_std",
] + cols[2:-4]
base_results = base_results[cols]

# Drop duplicates columns
base_results = base_results.loc[:,~base_results.columns.duplicated()]

base_results.sample(n=2)

dataset   model config_name  metric  metric_std  model_size  \
12476  image_segmentation  wisard        Dict  0.5159      0.0112  13169.3333   
1585                 iris  wisard        Dict  0.7852      0.0105    982.6667   

       model_size_std  accuracy  accuracy_std     f1  f1_std    ties  \
12476        105.1422    0.5159        0.0112 0.5000  0.0173 17.0000   
1585          15.1070    0.7852        0.0105 0.7771  0.0110 14.0000   

       ties_std  tuple_size                   encoder  resolution  bleach  \
12476    2.1602     11.0000  distributive-thermometer     11.0000  4.0000   
1585     4.8990      5.0000               thermometer      5.0000 15.0000   

       rams per discriminator  dataset_size  features  num_classes  \
12476                 19.0000         32678        19            7   
1585                   4.0000         10389         4            3   

       train_size  test_size  
12476         168         42  
1585          105         45

In [9]:
# Filter results
base_results = base_results[base_results["model"].isin(["wisard", "knn", "random-forest", "svm", "mlp"])]
base_results = base_results[~base_results["dataset"].isin(["mnist", "olivetti", "sensorless_drive"])]
# base_results = base_results[~base_results["dataset"].isin(["olivetti", "sensorless_drive"])]
base_results.shape

(14187, 23)

## 2. Wisard has competitive results with SKLearn

Here we show that the dict-wisard has competitive performance with the
classical machine learning algorithms.

To do that, we plot the accuracy of the wisard and sklearn algorithms for each
dataset.

In [10]:
# Read results and filter bloom filter results
results = base_results.copy()
results = results[
    (results["model"] != "wisard") | (results["config_name"] == "Dict")
]
results.shape

(3581, 23)

In [11]:
# Create a dataframe with the best performance for each dataset and model
best_metric_df = (
    results.groupby(["dataset", "model"])
    .apply(lambda group: group.loc[group["metric"].idxmax()])
    .reset_index(drop=True)
)

best_metric_df.value_counts("model")

model
knn              15
mlp              15
random-forest    15
svm              15
wisard           15
dtype: int64

In [12]:
# Assume 'results' DataFrame with columns: 'dataset', 'model', 'accuracy'

# Create a grouped bar chart for accuracy per model and dataset
fig_grouped_bar = px.bar(
    best_metric_df,
    x="dataset",
    y="metric",
    error_y="metric_std",
    color="model",
    #  title='Metric Comparison by Model and Dataset',
    labels={"metric": "Performance", "dataset": "Dataset", "model": ""},
    barmode="group",
    color_discrete_sequence=px.colors.qualitative.Prism,
)


# Display the plot
fig_grouped_bar.update_layout(
    legend=dict(
        orientation="h", yanchor="top", y=1.20, xanchor="center", x=0.5
    ),
    height=400,
    width=2480 / 2.5,
    font=dict(family="Times New Roman", size=14),
)

write_figure("models_performance.pdf", fig_grouped_bar)
fig_grouped_bar.show()

Figure written to: figures/models_performance.pdf
Filename   : models_performance.pdf
Latex label: models_performance


In [13]:
# Assume 'results' DataFrame with columns: 'dataset', 'model', 'accuracy'

# Create a grouped horizontal bar chart for accuracy per model and dataset with reversed bar groups
fig_grouped_bar = px.bar(
    best_metric_df,
    y="dataset",
    x="metric",
    error_x="metric_std",
    color="model",
    #  title='Metric Comparison by Model and Dataset',
    labels={"metric": "Performance", "dataset": "Dataset", "model": ""},
    barmode="group",
    orientation='h',
    color_discrete_sequence=px.colors.qualitative.Prism,
)


# Reverse the order of the bar groups
fig_grouped_bar.update_layout(
    yaxis=dict(autorange="reversed"),
)

# Display the plot
fig_grouped_bar.update_layout(
    legend=dict(
        orientation="h", yanchor="top", y=1.05, xanchor="center", x=0.5
    ),
    height=1200,
    width=2480 / 4,
    font=dict(family="Times New Roman", size=14),
)

write_figure("models_performance_horizontal.pdf", fig_grouped_bar)
fig_grouped_bar.show()

Figure written to: figures/models_performance_horizontal.pdf
Filename   : models_performance_horizontal.pdf
Latex label: models_performance_horizontal


In [14]:
# Count how many times per dataset, wisard is the best model

print("How many times per dataset, each model is the best?")
best_metric_df.loc[
    best_metric_df.groupby("dataset")["metric"].idxmax()
].value_counts("model").to_frame().reset_index()

How many times per dataset, each model is the best?


,model,0
0,random-forest,9
1,wisard,2
2,svm,2
3,knn,1
4,mlp,1


## 3. Wisard has competitive results with SKLearn and is smallest

Here we show that costing up to 2% of performance, the dict-wisard is much
smaller than the sklearn algorithms.

In [15]:
# Up to 2% of accuracy loss
metric_threshold = 0.01

In [16]:
# Read results and filter bloom filter results
results = base_results.copy()
results = results[
    (results["model"] != "wisard") | (results["config_name"] == "Dict")
]
results.shape

(3581, 23)

In [17]:
# Create a dataframe with the best performance for each dataset and model
best_metric_dataset_model = (
    results.groupby(["dataset", "model"])
    .apply(lambda group: group.loc[group["metric"].idxmax()])
    .reset_index(drop=True)
)

best_metric_dataset_model.value_counts("model")

model
knn              15
mlp              15
random-forest    15
svm              15
wisard           15
dtype: int64

In [18]:
# Add the model_size_ratio column. This column is the ratio between the model
# size of each model and the model size of the best model for each dataset
temp = []

for dset_name, dset_df in best_metric_dataset_model.groupby("dataset"):
    best_row = dset_df.sort_values(by="metric", ascending=False).iloc[0]
    dset_df["model_size_ratio"] = dset_df["model_size"] / best_row["model_size"]
    # Min max normalization
    dset_df["normalized_model_size_ratio"] = (
        dset_df["model_size_ratio"] - dset_df["model_size_ratio"].min()
    ) / (dset_df["model_size_ratio"].max() - dset_df["model_size_ratio"].min())
    dset_df["best_tradeoff"] = False

    best_tradeoff = (
        dset_df[dset_df["metric"] >= best_row["metric"] - metric_threshold]
        .sort_values(by="normalized_model_size_ratio", ascending=True)
        .iloc[0]
    )
    dset_df.loc[best_tradeoff.name, "best_tradeoff"] = True

    temp.append(dset_df)

best_metric_dataset_model = pd.concat(temp)
best_metric_dataset_model.head(n=8)

dataset          model config_name  metric  metric_std    model_size  \
0  breast_cancer            knn              0.9179      0.0000    99380.0000   
1  breast_cancer            mlp              0.9103      0.0000    85156.0000   
2  breast_cancer  random-forest              0.9709      0.0000   283286.0000   
3  breast_cancer            svm              0.8731      0.0000    27227.0000   
4  breast_cancer         wisard        Dict  0.9628      0.0100    94920.6667   
5       dry_bean            knn              0.7175      0.0000  1481493.0000   
6       dry_bean            mlp              0.2251      0.1470    63218.3333   
7       dry_bean  random-forest              0.9198      0.0017 14189404.0000   

   model_size_std  accuracy  accuracy_std     f1  f1_std   ties  ties_std  \
0          0.0000    0.9181        0.0000 0.9179  0.0000    NaN       NaN   
1          0.0000    0.9123        0.0000 0.9103  0.0000    NaN       NaN   
2          0.0000    0.9708        0.0000 0.9709  0.0000    NaN       NaN   
3          0.0000    0.8772        0.0000 0.8731  0.0000    NaN       NaN   
4       2226.8169    0.9630        0.0099 0.9628  0.0100 5.3333    1.2472   
5          0.0000    0.7216        0.0000 0.7175  0.0000    NaN       NaN   
6        219.9780    0.3286        0.0946 0.2251  0.1470    NaN       NaN   
7      54370.3571    0.9199        0.0017 0.9198  0.0017    NaN       NaN   

   tuple_size                   encoder  resolution  bleach  \
0         NaN                       NaN         NaN     NaN   
1         NaN                       NaN         NaN     NaN   
2         NaN                       NaN         NaN     NaN   
3         NaN                       NaN         NaN     NaN   
4     26.0000  distributive-thermometer     26.0000  4.0000   
5         NaN                       NaN         NaN     NaN   
6         NaN                       NaN         NaN     NaN   
7         NaN                       NaN         NaN     NaN   

   rams per discriminator  dataset_size  features  num_classes  train_size  \
0                     NaN        141416        30            3         398   
1                     NaN        141416        30            3         398   
2                     NaN        141416        30            3         398   
3                     NaN        141416        30            3         398   
4                 30.0000        141416        30            3         398   
5                     NaN       1773910        16            7       10888   
6                     NaN       1773910        16            7       10888   
7                     NaN       1773910        16            7       10888   

   test_size  model_size_ratio  normalized_model_size_ratio  best_tradeoff  
0        171            0.3508                       0.2818          False  
1        171            0.3006                       0.2262          False  
2        171            1.0000                       1.0000          False  
3        171            0.0961                       0.0000          False  
4        171            0.3351                       0.2644           True  
5       2723            0.1044                       0.1004          False  
6       2723            0.0045                       0.0000          False  
7       2723            1.0000                       1.0000           True

In [19]:
# pd.set_option('display.max_rows', 100)
dfs = {}
for dset_name, dset_df in best_metric_dataset_model.groupby("dataset"):
    max_val = dset_df["metric"].max()
    max_size =  dset_df["model_size"].max()
    dset_df["model_size"] = (dset_df["model_size"] / 1024)
    dset_df["relative performance"] = dset_df["metric"]  / max_val
    dset_df["relative size"] = dset_df["model_size"]  / max_size
    # dset_df.index = dset_df["model"]
    dset_df = dset_df[["model", "metric",  "model_size", "relative performance", "relative size", "accuracy", "f1"]]
    dfs[dset_name] = dset_df
    
result_df = pd.concat(dfs.values(), keys=dfs.keys())
result_df.reset_index(level=0, inplace=True)
result_df = result_df.rename(columns={"level_0": "dataset"})
result_df.to_csv("temp.csv", index=False)
print(f"Results written to temp.csv")
result_df

Results written to temp.csv


,dataset,model,metric,model_size,relative performance,relative size,accuracy,f1
0,breast_cancer,knn,0.9179,97.0508,0.9454,0.0003,0.9181,0.9179
1,breast_cancer,mlp,0.9103,83.1602,0.9376,0.0003,0.9123,0.9103
2,breast_cancer,random-forest,0.9709,276.6465,1.0000,0.0010,0.9708,0.9709
3,breast_cancer,svm,0.8731,26.5889,0.8992,0.0001,0.8772,0.8731
4,breast_cancer,wisard,0.9628,92.6960,0.9917,0.0003,0.9630,0.9628
...,...,...,...,...,...,...,...,...
70,yeast,knn,0.5464,178.0469,0.8802,0.0000,0.5522,0.5464
71,yeast,mlp,0.5896,52.9580,0.9498,0.0000,0.6027,0.5896
72,yeast,random-forest,0.6207,10533.5645,1.0000,0.0010,0.6296,0.6207
73,yeast,svm,0.5992,137.1494,0.9653,0.0000,0.6094,0.5992


In [20]:
# # Pivot the DataFrame to create the raw metric table
# table_df_raw = result_df.pivot(index='dataset', columns='model', values='metric')

# # Pivot the DataFrame to create the relative performance metric table
# table_df_relative_performance = result_df.pivot(index='dataset', columns='model', values='relative performance')

# # Join the two tables based on dataset
# joined_df = table_df_raw.join(table_df_relative_performance, lsuffix="_raw", rsuffix="_relative_performance")


# # joined_df.to_csv("temp.csv", index=True)

# # joined_df = joined_df.reset_index()

# joined_df = joined_df.rename_axis(None, axis=1).reset_index()
# # joined_df.index = range(len(joined_df))

# # joined_df.columns = joined_df.columns.to_list()


# raw_df = joined_df[['dataset', 'knn_raw', 'mlp_raw', 'random-forest_raw', 'svm_raw', 'wisard_raw']]
# relative_df = joined_df[['dataset', 'knn_relative_performance', 'mlp_relative_performance', 'random-forest_relative_performance', 'svm_relative_performance', 'wisard_relative_performance']]

# raw_df["dataset"] = raw_df["dataset"].str.replace("_", " ")
# relative_df["dataset"] = relative_df["dataset"].str.replace("_", " ")
# raw_df.columns = raw_df.columns.str.replace("-", " ")
# relative_df.columns = relative_df.columns.str.replace("-", " ")

# line = {"dataset": "Mean"}
# for c in raw_df.columns:
#     if c != "dataset":
#         line[c] = raw_df[c].mean()
# raw_df.loc[len(raw_df)] = line

# line = {"dataset": "Mean"}
# for c in relative_df.columns:
#     if c != "dataset":
#         line[c] = relative_df[c].mean()
# relative_df.loc[len(relative_df)] = line


# # Setting the dataset column as the index
# raw_df.set_index('dataset', inplace=True)
# relative_df.set_index('dataset', inplace=True)

# # # Concatenating the DataFrames
# final_df = pd.concat([raw_df, relative_df], axis=1)

# # # Renaming the columns
# final_df.columns = pd.MultiIndex.from_product([['Raw', 'Relative'], raw_df.columns.str.split('_').str[0]])

# write_latex_table("performance_table.tex", final_df.to_latex(float_format="%.2f"))
# final_df

In [21]:
# Pivot the DataFrame to create the raw metric table
table_df_raw = result_df.pivot(index='dataset', columns='model', values='metric')

# Pivot the DataFrame to create the relative performance metric table
table_df_relative_performance = result_df.pivot(index='dataset', columns='model', values='relative performance')

# Join the two tables based on dataset
joined_df = table_df_raw.join(table_df_relative_performance, lsuffix="_raw", rsuffix="_relative_performance")


# joined_df.to_csv("temp.csv", index=True)

# joined_df = joined_df.reset_index()

joined_df = joined_df.rename_axis(None, axis=1).reset_index()
# joined_df.index = range(len(joined_df))

# joined_df.columns = joined_df.columns.to_list()


raw_df = joined_df[['dataset', 'knn_raw', 'mlp_raw', 'random-forest_raw', 'svm_raw', 'wisard_raw']]
relative_df = joined_df[['dataset', 'knn_relative_performance', 'mlp_relative_performance', 'random-forest_relative_performance', 'svm_relative_performance', 'wisard_relative_performance']]

raw_df["dataset"] = raw_df["dataset"].str.replace("_", " ")
relative_df["dataset"] = relative_df["dataset"].str.replace("_", " ")
raw_df.columns = raw_df.columns.str.replace("-", " ")
relative_df.columns = relative_df.columns.str.replace("-", " ")

line = {"dataset": "Mean"}
for c in raw_df.columns:
    if c != "dataset":
        line[c] = raw_df[c].mean()
raw_df.loc[len(raw_df)] = line

line = {"dataset": "Mean"}
for c in relative_df.columns:
    if c != "dataset":
        line[c] = relative_df[c].mean()
relative_df.loc[len(relative_df)] = line


# Setting the dataset column as the index
raw_df.set_index('dataset', inplace=True)
relative_df.set_index('dataset', inplace=True)

# # Concatenating the DataFrames
final_df = pd.concat([raw_df, relative_df], axis=1)

# # Renaming the columns
final_df.columns = pd.MultiIndex.from_product([['Raw', 'Relative'], raw_df.columns.str.split('_').str[0]])

write_latex_table("performance_table.tex", final_df.to_latex(float_format="%.2f"))
final_df

Table written to: tables/performance_table.tex


/tmp/ipykernel_361841/328502811.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_361841/328502811.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_361841/328502811.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_361841/328502811.py:39: SettingWithCopyWarning:


A val

Raw                                    Relative         \
                       knn    mlp random forest    svm wisard      knn    mlp   
dataset                                                                         
breast cancer       0.9179 0.9103        0.9709 0.8731 0.9628   0.9454 0.9376   
dry bean            0.7175 0.2251        0.9198 0.6014 0.9024   0.7800 0.2447   
glass               0.7093 0.4554        0.7060 0.1850 0.6615   1.0000 0.6421   
image segmentation  0.5476 0.5556        0.6190 0.5000 0.8571   0.6389 0.6481   
iris                0.9556 0.9778        0.9333 1.0000 1.0000   0.9556 0.9778   
letter              0.9455 0.9205        0.9607 0.9200 0.9001   0.9842 0.9581   
motion sense        0.5049 0.6565        0.8078 0.5382 0.6500   0.6250 0.8127   
optical handwritten 0.9884 0.9810        0.9828 0.9920 0.9780   0.9964 0.9889   
rice                0.8637 0.4777        0.9153 0.8672 0.8994   0.9437 0.5220   
satimage            0.9025 0.9116        0.9106 0.8925 0.8867   0.9900 1.0000   
segment             0.9336 0.9423        0.9716 0.8701 0.8807   0.9609 0.9698   
sepsis              0.8910 0.8909        0.8908 0.8909 0.8916   0.9993 0.9992   
vehicle             0.8176 0.8176        0.9686 0.7176 0.8941   0.8441 0.8441   
wine                0.6714 0.8884        0.9816 0.6486 0.9629   0.6840 0.9051   
yeast               0.5464 0.5896        0.6207 0.5992 0.5700   0.8802 0.9498   
Mean                0.7942 0.7467        0.8773 0.7397 0.8598   0.8818 0.8267   

                                                 
                    random forest    svm wisard  
dataset                                          
breast cancer              1.0000 0.8992 0.9917  
dry bean                   1.0000 0.6538 0.9810  
glass                      0.9954 0.2608 0.9327  
image segmentation         0.7222 0.5833 1.0000  
iris                       0.9333 1.0000 1.0000  
letter                     1.0000 0.9576 0.9369  
motion sense               1.0000 0.6663 0.8046  
optical handwritten        0.9907 1.0000 0.9859  
rice                       1.0000 0.9475 0.9827  
satimage                   0.9989 0.9791 0.9726  
segment                    1.0000 0.8955 0.9064  
sepsis                     0.9991 0.9992 1.0000  
vehicle                    1.0000 0.7409 0.9231  
wine                       1.0000 0.6608 0.9810  
yeast                      1.0000 0.9653 0.9183  
Mean                       0.9760 0.8140 0.9545

In [21]:
def add_mean_line(df):
    line = {"dataset": "Mean"}
    for c in df.columns:
        if c != "dataset":
            line[c] = df[c].mean()
    df.loc[len(df)] = line
    return df

def raw_relative_table(df, raw_metric, relative_metric):
    # Pivot the DataFrame to create the raw metric table and relative table
    raw_df = (
        result_df.pivot(index="dataset", columns="model", values=raw_metric)
        .rename_axis(None, axis=1)
        .reset_index()
    )
    raw_df["dataset"] = raw_df["dataset"].str.replace("_", " ")
    raw_df.columns = raw_df.columns.str.replace("-", " ")
    raw_df = add_mean_line(raw_df)
    raw_df.set_index("dataset", inplace=True)

    relative_df = (
        result_df.pivot(
            index="dataset", columns="model", values=relative_metric
        )
        .rename_axis(None, axis=1)
        .reset_index()
    )

    relative_df["dataset"] = relative_df["dataset"].str.replace("_", " ")
    relative_df.columns = relative_df.columns.str.replace("-", " ")
    relative_df = add_mean_line(relative_df)
    relative_df.set_index("dataset", inplace=True)
    
    # Concatenating the DataFrames
    final_df = pd.concat([raw_df, relative_df], axis=1)

    final_df.columns = pd.MultiIndex.from_product(
        [["Raw", "Relative"], raw_df.columns.str.split("_").str[0]]
    )
    return final_df


performance_df = raw_relative_table(result_df, "metric", "relative performance")
write_latex_table("performance_table.tex", performance_df.to_latex(float_format="%.2f"))

size_df = raw_relative_table(result_df, "model_size", "relative size")
write_latex_table("size_table.tex", size_df.to_latex(float_format="%.2f"))

Table written to: tables/performance_table.tex
Table written to: tables/size_table.tex


In [22]:
size_df

Raw                                              \
                          knn      mlp random forest       svm     wisard   
dataset                                                                     
breast cancer         97.0508  83.1602      276.6465   26.5889    92.6960   
dry bean            1446.7705  61.7367    13856.8398 1468.3936   455.0872   
glass                 25.7041  45.7988      721.6768   18.3955    66.1410   
image segmentation    26.9375  71.6204      485.3838   28.9463   190.3506   
iris                   9.6777  26.9551      131.2373    3.6074     1.3398   
letter              1860.2314 109.7171   101780.5475 2666.9131 10137.4313   
motion sense        9629.2402 868.4893     9218.5420 8445.1621 32483.6113   
optical handwritten 2283.8477 182.6178    11563.1270  715.9619 17175.9730   
rice                 400.0762  26.0788     2801.4219   73.6484     1.4554   
satimage            1301.7617 109.0924     8015.7311  475.2129  5362.9385   
segment              253.3555  70.7018     1730.8545  165.6230   117.8405   
sepsis              5838.1943  16.7725     8872.0020  560.6406    33.5684   
vehicle              101.0117  57.7461      969.3932   82.1152   291.2301   
wine                  29.9619  48.0498      185.1348   11.0322    75.8529   
yeast                178.0469  52.9580    10533.5645  137.1494   463.6279   
Mean                1565.4579 122.0997    11409.4735  991.9594  4463.2763   

                    Relative                                     
                         knn    mlp random forest    svm wisard  
dataset                                                          
breast cancer         0.0003 0.0003        0.0010 0.0001 0.0003  
dry bean              0.0001 0.0000        0.0010 0.0001 0.0000  
glass                 0.0000 0.0001        0.0010 0.0000 0.0001  
image segmentation    0.0001 0.0001        0.0010 0.0001 0.0004  
iris                  0.0001 0.0002        0.0010 0.0000 0.0000  
letter                0.0000 0.0000        0.0010 0.0000 0.0001  
motion sense          0.0003 0.0000        0.0003 0.0003 0.0010  
optical handwritten   0.0001 0.0000        0.0007 0.0000 0.0010  
rice                  0.0001 0.0000        0.0010 0.0000 0.0000  
satimage              0.0002 0.0000        0.0010 0.0001 0.0007  
segment               0.0001 0.0000        0.0010 0.0001 0.0001  
sepsis                0.0006 0.0000        0.0010 0.0001 0.0000  
vehicle               0.0001 0.0001        0.0010 0.0001 0.0003  
wine                  0.0002 0.0003        0.0010 0.0001 0.0004  
yeast                 0.0000 0.0000        0.0010 0.0000 0.0000  
Mean                  0.0002 0.0001        0.0009 0.0001 0.0003

In [23]:
dfs = []

def belongs_to_pareto(df, model):
    metric = df[df["model"] == model]["metric"].iloc[0]
    size = df[df["model"] == model]["model_size"].iloc[0]
    
    for r_index, row in df.iterrows():
        if row["metric"] > metric and row["model_size"] < size:
            return False
        
    return True
    
columns = result_df["dataset"].unique()
    

models = ["wisard", "random-forest", "svm", "mlp", "knn"]
for c in columns:
    x_df = result_df[result_df["dataset"] == c]
    if belongs_to_pareto(x_df, "wisard"):
        print(f"Wisard belongs to pareto in {c}")
        # x_df.index = x_df.index.str.upper()
    
    lines = []
    for m in models:
        line = x_df[x_df["model"] == m]
        if belongs_to_pareto(x_df, m):
            print(f"Wisard belongs to pareto in {c}")
            line["pareto"] = True
        else:
            line["pareto"] = False
        lines.append(line)
        
        
    x_df = pd.concat(lines)
    
    dfs.append(x_df)
    
n = pd.concat(dfs).reset_index()
    
# n["dataset"] = n["level_0"]
# n["pareto"] = n["pareto"].astype(int)
n

Wisard belongs to pareto in breast_cancer
Wisard belongs to pareto in breast_cancer
Wisard belongs to pareto in breast_cancer
Wisard belongs to pareto in breast_cancer
Wisard belongs to pareto in breast_cancer
Wisard belongs to pareto in dry_bean
Wisard belongs to pareto in dry_bean
Wisard belongs to pareto in dry_bean
Wisard belongs to pareto in dry_bean
Wisard belongs to pareto in glass
Wisard belongs to pareto in glass
Wisard belongs to pareto in image_segmentation
Wisard belongs to pareto in image_segmentation
Wisard belongs to pareto in image_segmentation
Wisard belongs to pareto in image_segmentation
Wisard belongs to pareto in iris
Wisard belongs to pareto in iris
Wisard belongs to pareto in iris
Wisard belongs to pareto in letter
Wisard belongs to pareto in letter
Wisard belongs to pareto in letter
Wisard belongs to pareto in motion_sense
Wisard belongs to pareto in motion_sense
Wisard belongs to pareto in optical_handwritten
Wisard belongs to pareto in optical_handwritten
Wisa

/tmp/ipykernel_361841/387624777.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_361841/387624777.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_361841/387624777.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne

,index,dataset,model,metric,model_size,relative performance,relative size,accuracy,f1,pareto
0,4,breast_cancer,wisard,0.9628,92.6960,0.9917,0.0003,0.9630,0.9628,True
1,2,breast_cancer,random-forest,0.9709,276.6465,1.0000,0.0010,0.9708,0.9709,True
2,3,breast_cancer,svm,0.8731,26.5889,0.8992,0.0001,0.8772,0.8731,True
3,1,breast_cancer,mlp,0.9103,83.1602,0.9376,0.0003,0.9123,0.9103,True
4,0,breast_cancer,knn,0.9179,97.0508,0.9454,0.0003,0.9181,0.9179,False
...,...,...,...,...,...,...,...,...,...,...
70,74,yeast,wisard,0.5700,463.6279,0.9183,0.0000,0.5825,0.5700,False
71,72,yeast,random-forest,0.6207,10533.5645,1.0000,0.0010,0.6296,0.6207,True
72,73,yeast,svm,0.5992,137.1494,0.9653,0.0000,0.6094,0.5992,True
73,71,yeast,mlp,0.5896,52.9580,0.9498,0.0000,0.6027,0.5896,True


In [25]:
# Assume 'results' DataFrame with columns: 'dataset', 'model', 'accuracy'

# Create a grouped horizontal bar chart for accuracy per model and dataset with reversed bar groups
fig_grouped_bar = px.bar(
    n,
    y="dataset",
    x="metric",
    # error_x="metric_std",
    color="model",
    #  title='Metric Comparison by Model and Dataset',
    labels={"metric": "Performance", "dataset": "Dataset", "model": ""},
    barmode="group",
    orientation='h',
    color_discrete_sequence=px.colors.qualitative.Prism,
)


# Reverse the order of the bar groups
fig_grouped_bar.update_layout(
    yaxis=dict(autorange="reversed"),
)

# Display the plot
fig_grouped_bar.update_layout(
    legend=dict(
        orientation="h", yanchor="top", y=1.05, xanchor="center", x=0.5
    ),
    height=1200,
    width=2480 / 4,
    font=dict(family="Times New Roman", size=14),
)

write_figure("models_performance_horizontal.pdf", fig_grouped_bar)
fig_grouped_bar.show()

Figure written to: figures/models_performance_horizontal.pdf
Filename   : models_performance_horizontal.pdf
Latex label: models_performance_horizontal


In [42]:
data = result_df[result_df.dataset == "iris"]

import plotly.express as px
import plotly.graph_objs as go

# Define marker symbols for each model
marker_symbols = {
    "knn": "circle",
    "mlp": "square",
    "random-forest": "diamond",
    "svm": "cross",
    "wisard": "x"
}

# Create trace
traces = []
for model in data['model'].unique():
    model_data = data[data['model'] == model]
    trace = go.Scatter(
        x=model_data['relative size'],
        y=model_data['metric'],
        mode='markers',
        text=model_data['model'],
        marker=dict(
            symbol=marker_symbols[model],
            size=12,
            opacity=0.8,
            color=px.co
        ),
        name=model
    )
    traces.append(trace)

# Create layout
layout = go.Layout(
    title='Relative Size vs. Metric',
    xaxis=dict(title='Relative Size'),
    yaxis=dict(title='Metric'),
    hovermode='closest'
)

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show plot
fig.show()



In [30]:
fig = px.scatter(result_df.reset_index(), y="metric", x="relative size", symbol="model", facet_col="dataset", facet_col_wrap=4)
fig.update_layout(
    legend=dict(
        orientation="h", yanchor="top", y=1.05, xanchor="center", x=0.5
    ),
    height=1200,
    width=2480,
    font=dict(family="Times New Roman", size=14),
)
fig.show()

In [ ]:
dfs = {}
for dset_name, dset_df in best_metric_dataset_model.groupby("dataset"):
    max_val = dset_df["metric"].max()
    dset_df["model_size (KB)"] = (dset_df["model_size"] / 1024)
    dset_df["relative performance"] = dset_df["metric"]  / max_val
    dset_df = dset_df[["relative performance", "model_size (KB)"]]
    dfs[dset_name] = dset_df
    
result_df = pd.concat(dfs.values(), keys=dfs.keys()).reset_index()
# result_df.reset_index(level=0, inplace=True)
result_df

In [ ]:
print("How many times per dataset, each model is the best tradeoff?")
best_metric_dataset_model[
    best_metric_dataset_model.best_tradeoff == True
].value_counts("model").to_frame().reset_index()

In [ ]:
facets = 3
cmap = px.colors.qualitative.Prism

colors = {
    name: cmap[i]
    for i, name in enumerate(sorted(best_metric_dataset_model.model.unique()))
}

# Scatter plot for trade-off with normalized model size
fig_tradeoff_normalized = px.scatter(
    best_metric_dataset_model,
    x="metric",
    y="normalized_model_size_ratio",
    color="model",
    facet_col="dataset",
    facet_col_wrap=facets,
    # title="Trade-off Between Metric and Normalized Model Size Across Datasets",
    labels={
        "metric": "Performance",
        "normalized_model_size_ratio": "Size Ratio (normalized)",
        "model": "",
    },
    facet_row_spacing=0.07,
    width=1400,
    height=800,
    color_discrete_sequence=cmap,
)


fig_tradeoff_normalized.update_traces(
    marker=dict(size=7.5),
)

fig_tradeoff_normalized.update_xaxes(showticklabels=True)

fig_tradeoff_normalized.update_yaxes(showticklabels=True)

fig_tradeoff_normalized.for_each_annotation(
    lambda a: a.update(text=a.text.split("=")[-1])
)

# fig_tradeoff_normalized.update_layout(
#     legend=dict(
#         orientation="h", yanchor="bottom", y=-0.15, xanchor="center", x=0.5,
#     ),

# )

fig_tradeoff_normalized.update_layout(
    legend=dict(
        orientation="h",
        yanchor="top",
        y=1.05,
        xanchor="center",
        x=0.5,
        itemsizing="constant",  # Set this to "constant" to show only one item for color
        font=dict(family="Times New Roman", size=14),
    ),
    height=1200,
    width=2480 /2.5,
    font=dict(family="Times New Roman", size=14),
)
num_rows = len(results.dataset.unique()) // facets
num_cols = facets
# datasets = list(sorted(results.dataset.unique(), reverse=False))

for r in range(num_rows):
    for c in range(num_cols):
       
        facet = r * num_cols + c
        dset = fig_tradeoff_normalized.layout.annotations[facet]['text']

        x_line = (
            results[results["dataset"] == dset]["metric"].max()
            - metric_threshold
        )
        
 # Scatter plot for best tradeoff points with a cross
        best_tradeoff_points = best_metric_dataset_model[
            (best_metric_dataset_model["dataset"] == dset)
            & (best_metric_dataset_model["best_tradeoff"] == True)
        ]
        
        best_tradeoff_model = best_tradeoff_points.model.iloc[0]
        
        # print(f"Facet {facet}: {dset} - {best_tradeoff_model}. Max: {results[results['dataset'] == dset]['metric'].max()}, line: {x_line}")
        
        fig_tradeoff_normalized.add_trace(
            go.Scatter(
                x=best_tradeoff_points["metric"],
                y=best_tradeoff_points["normalized_model_size_ratio"],
                mode="markers",
                marker=dict(
                    size=10,
                    symbol="x",
                    color=colors[best_tradeoff_model]
                ),
                showlegend=False,  # To not duplicate in the legend
            ),
            row=r + 1,
            col=c + 1,
        )
        
        fig_tradeoff_normalized.add_vline(
            x=x_line, line_dash="dot", row=r + 1, col=c + 1, line_width=1
        )

# Display the plot
fig_tradeoff_normalized.show()

write_figure("performance_size_tradeoff_normalized.pdf", fig_tradeoff_normalized)

In [ ]:
best_metric_dataset_model[
    best_metric_dataset_model.best_tradeoff == True
][["dataset", "model", "config_name", "metric", "model_size", "model_size_ratio"]]

In [ ]:
best_metric_dataset_model[best_metric_dataset_model ["dataset"] == "iris"].dropna(axis=1)[["model", "metric", "model_size", "model_size_ratio", "normalized_model_size_ratio"]]

In [ ]:
best_metric_dataset_model[["dataset", "model", "config_name", "metric", "model_size", "model_size_ratio"]]

In [ ]:
best_metric_dataset_model[
    (best_metric_dataset_model.best_tradeoff == True) & (best_metric_dataset_model.model == "wisard")
]["model_size_ratio"]

In [ ]:
encoder_info = best_metric_dataset_model[best_metric_dataset_model.model == "wisard"]
encoder_info = encoder_info[["dataset", "encoder", "resolution", "tuple_size", "bleach"]].reset_index(drop=True)
encoder_info.rename(columns={"encoder": "Encoder", "resolution": "Resolution", "tuple_size": "Tuple Size", "bleach": "Bleach"}, inplace=True)
encoder_info["Encoder"] = encoder_info["Encoder"].apply(lambda x: "Distributive Thermometer" if x == "distributive-thermometer" else "Thermometer")
encoder_info["Resolution"] = encoder_info["Resolution"].astype(int)
encoder_info["Tuple Size"] = encoder_info["Tuple Size"].astype(int)
encoder_info["Bleach"] = encoder_info["Bleach"].astype(int)
encoder_info

In [ ]:
latex_str = encoder_info.to_latex(
    index=False,
    escape=True,
    caption="Parameters used for each experiment",
    label="tab:experiment-parameters",
    float_format="%.2f",
)
write_latex_table("experiment_parameters.tex", latex_str)

## 4. Select the wisard with best bloom filter

Here we show that, costing up to 1% of performance of the best dict-wisard, a 
space-efficient bloom filter achieves the best results.

In [ ]:
# Up to 1% of accuracy loss
metric_threshold = 0.01

In [ ]:
results = base_results.copy()
best_results = best_metric_dataset_model.copy()

results = results[results["model"] == "wisard"]
best_results = best_results[best_results["model"] == "wisard"]

results["bloom-filter"] = results["config_name"].apply(lambda x: x.split(" ")[0])
best_results["bloom-filter"] = best_results["config_name"].apply(lambda x: x.split(" ")[0])

In [ ]:
temp = []


for dset_name, dset_df in results.groupby("dataset"):
    best_model = best_results[best_results["dataset"] == dset_name].iloc[0]
    # print(f"*** Dataset: {dset_name} with metric: {best_model['metric']} and model size: {int(best_model['model_size'])}")
    bests = dset_df[dset_df["metric"] >= best_model["metric"] - metric_threshold]
    bests["metric_improvement"] = bests["metric"]/ best_model["metric"]
    bests["model_size_improvement"] = bests["model_size"]/ best_model["model_size"]
    bests["best_metric"]  = best_model["metric"]
    bests["best_model_size"]  = best_model["model_size"]
    temp.append(bests)
    
results = pd.concat(temp, ignore_index=True)
results.sample(n=3)

In [ ]:
best_results = results.loc[results.groupby("dataset")["model_size_improvement"].idxmin()]
best_results["bloom-filter"].value_counts()

In [ ]:
best_results_table = best_results.sort_values(by=["dataset"])[["dataset", "config_name", "model_size_improvement"]]

best_results_table.rename(
    columns={
        "dataset": "Dataset",
        "config_name": "Bloom Filter",
        "model_size_improvement": "Size Ratio",
    },
    inplace=True,
)

best_results_table

In [ ]:
latex_str = best_results_table.to_latex(
    index=False,
    escape=True,
    caption="Best Bloom Filter configuration for each dataset",
    label="tab:best_bloom_filter",
    float_format="%.2f",
)

write_latex_table("best_bloom_filter.tex", latex_str)

In [ ]:
best_results_table = best_results.sort_values(by=["dataset"])[["dataset", "bloom-filter", "config_name", "model_size_improvement"]]
best_results_table.groupby("bloom-filter")["model_size_improvement"].agg(["mean", "std", "count"])

# 5. Select the wisard with best aggregated bloom filter

Here we show that, costing up to 1% of performance of the best dict-wisard, a 
space-efficient bloom filter achieves the best results.

In [ ]:
# Up to 2% of accuracy loss
metric_threshold = 0.01

In [ ]:
results = base_results.copy()
best_results = best_metric_dataset_model.copy()

results = results[results["model"] == "wisard"]
best_results = best_results[best_results["model"] == "wisard"]

results["bloom-filter"] = results["config_name"].apply(lambda x: x.split(" ")[0])
best_results["bloom-filter"] = best_results["config_name"].apply(lambda x: x.split(" ")[0])

# Filter: only Dict, CountingBloomFilter and CountMinSketch
results = results[results["bloom-filter"].isin(["Dict", "CountingBloomFilter", "CountMinSketch"])]
best_results = best_results[best_results["bloom-filter"].isin(["Dict", "CountingBloomFilter", "CountMinSketch"])]

In [ ]:
temp = []


for dset_name, dset_df in results.groupby("dataset"):
    best_model = best_results[best_results["dataset"] == dset_name].iloc[0]
    # print(f"*** Dataset: {dset_name} with metric: {best_model['metric']} and model size: {int(best_model['model_size'])}")
    bests = dset_df[dset_df["metric"] >= best_model["metric"] - metric_threshold]
    bests["metric_improvement"] = bests["metric"]/ best_model["metric"]
    bests["model_size_improvement"] = bests["model_size"]/ best_model["model_size"]
    bests["best_metric"]  = best_model["metric"]
    bests["best_model_size"]  = best_model["model_size"]
    temp.append(bests)
    
results = pd.concat(temp, ignore_index=True)
results.sample(n=3)

In [ ]:
best_results = results.loc[results.groupby("dataset")["model_size_improvement"].idxmin()]
best_results["bloom-filter"].value_counts()

In [ ]:
best_results_table = best_results.sort_values(by=["dataset"])[["dataset", "config_name", "model_size_improvement"]]

best_results_table.rename(
    columns={
        "dataset": "Dataset",
        "config_name": "Bloom Filter Configuration",
        "model_size_improvement": "Size Ratio",
    },
    inplace=True,
)

best_results_table

In [ ]:
latex_str = best_results_table.to_latex(
    index=False,
    escape=True,
    caption="Best Bloom Filter configuration for each dataset",
    label="tab:best_bloom_filter_agg",
    float_format="%.2f",
)

write_latex_table("best_bloom_filter_agg.tex", latex_str)

In [ ]:
best_results_table = best_results.sort_values(by=["dataset"])[["dataset", "bloom-filter", "config_name", "model_size_improvement"]]
best_results_table.groupby("bloom-filter")["model_size_improvement"].agg(["mean", "std", "count"])